ModuleNotFoundError: No module named 'vqgan_with_pos_trial_v2'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
"""
In GroupNorm, channels are divided into groups, and the mean and 
variance are calculated within each group across all instances in a 
batch. This makes GroupNorm's performance less dependent on the batch 
size, and it can maintain its effectiveness even with a batch size of 1. 
This can be particularly useful in applications where memory constraints 
limit the maximum batch size, such as high-resolution image processing.

"""

class GroupNorm(nn.Module):
    def __init__(self, channels):
        super(GroupNorm, self).__init__()
        
        # num_grapups should be divisble by in_channels
        # for example 32 is divisible by 128.
        # 32 is divisible by 256
        # 32 is divisibble by 5122
        self.gn = nn.GroupNorm(num_groups=32, num_channels=channels, eps=1e-6, affine=True)

    def forward(self, x):
        return self.gn(x)


class Swish(nn.Module):
    
    def forward(self, x):
        
        return x * torch.sigmoid(x)


class ResidualBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels):
        
        super(ResidualBlock, self).__init__()
        
        self.in_channels = in_channels
        
        self.out_channels = out_channels
        
        self.block = nn.Sequential(
            GroupNorm(in_channels),
            Swish(),
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            GroupNorm(out_channels),
            Swish(),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1)  # this layer is used to preserve depth which is used in return x + self.block(x)
        )

        if in_channels != out_channels:
            self.channel_up = nn.Conv2d(in_channels, out_channels, 1, 1, 0)

    def forward(self, x):
        if self.in_channels != self.out_channels:
            return self.channel_up(x) + self.block(x)
        else:
            return x + self.block(x)


class UpSampleBlock(nn.Module):
    def __init__(self, channels):
        super(UpSampleBlock, self).__init__()
        self.conv = nn.Conv2d(channels, channels, 3, 1, 1)

    def forward(self, x):
        x = F.interpolate(x, scale_factor=2.0)
        return self.conv(x)

class PositionalEncoding2D(nn.Module):
    def __init__(self, d_model, max_len=2048, dropout_prob=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout_prob)
        
        pe = torch.zeros(max_len, max_len, d_model)
        position_i = torch.arange(max_len, dtype=torch.float).unsqueeze(1)
        position_j = torch.arange(max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2.) * -(math.log(10000.0) / d_model))

        pe[:, :, 0::2] = torch.sin(position_i * div_term)
        pe[:, :, 1::2] = torch.cos(position_j * div_term)

        pe = pe.permute(2, 0, 1).unsqueeze(0)  # make shape [1, C, H, W]

        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :, :x.size(2), :x.size(3)]
        return self.dropout(x)
    
class DownSampleBlock(nn.Module):
    def __init__(self, channels):
        super(DownSampleBlock, self).__init__()
        self.conv = nn.Conv2d(channels, channels, 3, 2, 0)

    def forward(self, x):
        pad = (0, 1, 0, 1)
        x = F.pad(x, pad, mode="constant", value=0)
        return self.conv(x)

# NonLocal NN: https://arxiv.org/pdf/1711.07971.pdf
class NonLocalBlock(nn.Module):
    def __init__(self, channels):
        super(NonLocalBlock, self).__init__()
        self.in_channels = channels

        self.gn = GroupNorm(channels)
        self.q = nn.Conv2d(channels, channels, 1, 1, 0)
        self.k = nn.Conv2d(channels, channels, 1, 1, 0)
        self.v = nn.Conv2d(channels, channels, 1, 1, 0)
        self.proj_out = nn.Conv2d(channels, channels, 1, 1, 0)

    def forward(self, x):
        h_ = self.gn(x)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        b, c, h, w = q.shape

        q = q.reshape(b, c, h*w)
        q = q.permute(0, 2, 1)
        k = k.reshape(b, c, h*w)
        v = v.reshape(b, c, h*w)

        attn = torch.bmm(q, k)
        attn = attn * (int(c)**(-0.5))
        attn = F.softmax(attn, dim=2)
        attn = attn.permute(0, 2, 1)

        A = torch.bmm(v, attn)
        A = A.reshape(b, c, h, w)

        return x + A














In [2]:
import torch.nn as nn
import os
# from helper import ResidualBlock, NonLocalBlock, DownSampleBlock, UpSampleBlock, GroupNorm, Swish

# from PositionalEncoding2dJupyter import PositionalEncoding2D

# from helperjupyter import ResidualBlock, NonLocalBlock, UpSampleBlock, GroupNorm, Swish,PositionalEncoding2D
# Encoder block ==> Page 11/52 (pdf): https://arxiv.org/pdf/2012.09841.pdf

"""_summary_

Step-by-step breakdown of the Encoder module:

Module 1 - Initial Convolution Layer

Layer 1: Conv2d layer (input: image_channels, output: channels[0], kernel_size: 3, stride: 1, padding: 1)
Module 2 - Residual Blocks and Non-local Blocks

For each pair of consecutive channels in the channels list:
Layer 2a: ResidualBlock (input: in_channels, output: out_channels)
Layer 2b: ResidualBlock (input: out_channels, output: out_channels)
If the current resolution is in attn_resolutions, add a NonLocalBlock (input: out_channels)
If not at the last pair of consecutive channels, add a DownSampleBlock (input: out_channels)

Module 3 - Final Layers

Layer 3a: ResidualBlock (input: channels[-1], output: channels[-1])
Layer 3b: NonLocalBlock (input: channels[-1])
Layer 3c: ResidualBlock (input: channels[-1], output: channels[-1])
Layer 3d: GroupNorm (input: channels[-1])
Layer 3e: Swish activation
Layer 3f: Conv2d layer (input: channels[-1], output: latent_dim, kernel_size: 3, stride: 1, padding: 1)
The entire architecture is then built using a nn.Sequential layer containing all the layers from Module 1, Module 2, and Module 3.

channels = [128, 128, 128, 256, 256, 512]
There are 5 instances of Module 2 for the given channel list.

Here's the breakdown:

Module 2.1: Transition from channels[0] (128) to channels[1] (128)
Module 2.2: Transition from channels[1] (128) to channels[2] (128)
Module 2.3: Transition from channels[2] (128) to channels[3] (256)
Module 2.4: Transition from channels[3] (256) to channels[4] (256)
Module 2.5: Transition from channels[4] (256) to channels[5] (512)

The NonLocalBlock is added based on the attn_resolutions list. 
In the given code, attn_resolutions = [16]. The purpose of using the NonLocalBlock is 
to model long-range dependencies within the input feature maps by capturing spatial relationships at specific resolutions.



Returns:
_type_: _description_
"""


# class Encoder(nn.Module):
#     def __init__(self, args):
#         super(Encoder, self).__init__()
#         channels = [128, 128, 128, 256, 256, 512]
#         attn_resolutions = [16]
#         num_res_blocks = 2
#         resolution = 256

#         layers = [nn.Conv2d(args.image_channels, channels[0], 3, 1, 1)]
#         layers.append(PositionalEncoding2D(channels[0], max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after first Conv2d
        
#         for i in range(len(channels)-1):
#             in_channels = channels[i]
#             out_channels = channels[i + 1]
#             for j in range(num_res_blocks):
#                 layers.append(ResidualBlock(in_channels, out_channels))
#                 layers.append(PositionalEncoding2D(out_channels, max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after ResidualBlock
#                 in_channels = out_channels
#                 if resolution in attn_resolutions:
#                     layers.append(NonLocalBlock(in_channels))
#                     layers.append(PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after NonLocalBlock
#             if i != len(channels)-2: 
#                 layers.append(DownSampleBlock(channels[i+1]))
#                 resolution //= 2
#         layers.append(ResidualBlock(channels[-1], channels[-1]))
#         layers.append(PositionalEncoding2D(channels[-1], max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after ResidualBlock
#         layers.append(NonLocalBlock(channels[-1]))
#         layers.append(PositionalEncoding2D(channels[-1], max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after NonLocalBlock
#         layers.append(ResidualBlock(channels[-1], channels[-1]))
#         layers.append(PositionalEncoding2D(channels[-1], max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after ResidualBlock
#         layers.append(GroupNorm(channels[-1]))
#         layers.append(Swish())
#         layers.append(nn.Conv2d(channels[-1], args.latent_dim, 3, 1, 1))
#         self.model = nn.Sequential(*layers)

#     def forward(self, x):
#         return self.model(x)

class Encoder(nn.Module):
    def __init__(self, args):
        super(Encoder, self).__init__()
        channels = [128, 128, 128, 256, 256, 512]
        attn_resolutions = [16]
        num_res_blocks = 2
        resolution = 256

        layers = [nn.Conv2d(args.image_channels, channels[0], 3, 1, 1)]
        layers.append(PositionalEncoding2D(channels[0], max_len=512, dropout_prob=0.5))  # PositionalEncoding2D at the start
        
        for i in range(len(channels)-1):
            in_channels = channels[i]
            out_channels = channels[i + 1]
            for j in range(num_res_blocks):
                layers.append(ResidualBlock(in_channels, out_channels))
                in_channels = out_channels
                if resolution in attn_resolutions:
                    layers.append(NonLocalBlock(in_channels))
            if i != len(channels)-2: 
                layers.append(DownSampleBlock(channels[i+1]))
                resolution //= 2
            
            # Adding PositionalEncoding2D in the middle of the encoder
            if i == len(channels) // 2:
                layers.append(PositionalEncoding2D(channels[i+1], max_len=512, dropout_prob=0.5))  

        layers.append(ResidualBlock(channels[-1], channels[-1]))
        layers.append(NonLocalBlock(channels[-1]))
        layers.append(ResidualBlock(channels[-1], channels[-1]))
        layers.append(GroupNorm(channels[-1]))
        layers.append(Swish())
        layers.append(nn.Conv2d(channels[-1], args.latent_dim, 3, 1, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [3]:
# import torch
# import torch.nn as nn

# # Your classes here: PositionalEncoding2D, ResidualBlock, NonLocalBlock, DownSampleBlock, GroupNorm, Swish

# class Args:
#     def __init__(self, latent_dim, image_channels, device):
#         self.latent_dim = latent_dim
#         self.image_channels = image_channels
#         self.device = device
# # 
# # class Encoder(nn.Module):
#     # your Encoder code

# # Set up the arguments
# args = Args(latent_dim=256, image_channels=3, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# # Initialize the Encoder
# encoder = Encoder(args)

# # Create a random tensor of size (batch_size, channels, height, width)
# x = torch.randn(5, 3, 256, 256)

# # Pass the tensor through the encoder
# try:
#     y = encoder(x)
#     print("Encoder executed successfully. Output shape:", y.shape)
# except IndexError as e:
#     print("Error occurred during execution of encoder:", str(e))

In [4]:
# class Decoder(nn.Module):
#     def __init__(self, args):
#         super(Decoder, self).__init__()
#         channels = [512, 256, 256, 128, 128]
#         attn_resolutions = [16]
#         num_res_blocks = 3
#         resolution = 16

#         in_channels = channels[0]
#         # first 4 layers of the decoder. 
#         layers = [nn.Conv2d(args.latent_dim, in_channels, 3, 1, 1),
#                   PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5),  # PositionalEncoding2D after first Conv2d
#                   ResidualBlock(in_channels, in_channels),
#                   PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5),  # PositionalEncoding2D after ResidualBlock
#                   NonLocalBlock(in_channels),
#                   PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5),  # PositionalEncoding2D after NonLocalBlock
#                   ResidualBlock(in_channels, in_channels),
#                   PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5)]  # PositionalEncoding2D after ResidualBlock

#         # all other layers. 
#         for i in range(len(channels)):
#             out_channels = channels[i]
#             for j in range(num_res_blocks):
#                 layers.append(ResidualBlock(in_channels, out_channels))
#                 layers.append(PositionalEncoding2D(out_channels, max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after ResidualBlock
#                 in_channels = out_channels
#                 if resolution in attn_resolutions:
#                     layers.append(NonLocalBlock(in_channels))
#                     layers.append(PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after NonLocalBlock
#             if i != 0:
#                 layers.append(UpSampleBlock(in_channels))
#                 resolution *= 2

#         layers.append(GroupNorm(in_channels))
#         layers.append(Swish())
#         layers.append(nn.Conv2d(in_channels, args.image_channels, 3, 1, 1))
#         self.model = nn.Sequential(*layers)

#     def forward(self, x):
#         return self.model(x)
    
    
class Decoder(nn.Module):
    def __init__(self, args):
        super(Decoder, self).__init__()
        channels = [512, 256, 256, 128, 128]
        attn_resolutions = [16]
        num_res_blocks = 3
        resolution = 16

        in_channels = channels[0]
        # first 4 layers of the decoder. 
        layers = [nn.Conv2d(args.latent_dim, in_channels, 3, 1, 1),
                  PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5)]  # PositionalEncoding2D after first Conv2d

        # Adding Residual Blocks, NonLocalBlock, and UpSampleBlocks
        for i in range(len(channels)):
            out_channels = channels[i]
            for j in range(num_res_blocks):
                layers.append(ResidualBlock(in_channels, out_channels))
                if i == len(channels) // 2 and j == num_res_blocks // 2:  # Approximately the middle of the network
                    layers.append(PositionalEncoding2D(out_channels, max_len=512, dropout_prob=0.5))  # PositionalEncoding2D
                in_channels = out_channels
                if resolution in attn_resolutions:
                    layers.append(NonLocalBlock(in_channels))
            if i != 0:
                layers.append(UpSampleBlock(in_channels))
                resolution *= 2

        layers.append(GroupNorm(in_channels))
        layers.append(Swish())
        layers.append(nn.Conv2d(in_channels, args.image_channels, 3, 1, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [5]:
# class Args:
#     def __init__(self):
#         self.latent_dim = 10
#         self.image_channels = 3

# args = Args()
# decoder = Decoder(args)

# # Create a random tensor with shape (batch_size, latent_dim, height, width)
# x = torch.randn(1, args.latent_dim, 16, 16)
# output = decoder(x)

# print(output.shape)  # Should print torch.Size([1

In [9]:
import torch
import torch.nn as nn
# from encoder_with_positional_encoding_v2 import Encoder
# from decoder_with_positional_encoding_v2 import Decoder
from codebook import Codebook
import math

class VQGAN(nn.Module):
    def __init__(self, args):
        super(VQGAN, self).__init__()
        self.encoder = Encoder(args).to(device=args.device)
        self.decoder = Decoder(args).to(device=args.device)
        self.codebook = Codebook(args).to(device=args.device)       
        
        # original vq-gan has prequantization and post quantization convolution layers.
        self.quant_conv = nn.Conv2d(args.latent_dim, args.latent_dim, 1).to(device=args.device)
        self.post_quant_conv = nn.Conv2d(args.latent_dim, args.latent_dim, 1).to(device=args.device)


    def forward(self, imgs):
        encoded_images = self.encoder(imgs)
        # print("encoded_images_shape", encoded_images.shape)
        # pre quantisation layer. 
        quant_conv_encoded_images = self.quant_conv(encoded_images)
        # print("quant_conv_encoded_images_shape",quant_conv_encoded_images.shape)
        
        codebook_mapping, codebook_indices, q_loss = self.codebook(quant_conv_encoded_images)
        # print("codebook_mapping_shape", codebook_mapping.shape)
        # post quantization layer.
        post_quant_conv_mapping = self.post_quant_conv(codebook_mapping)
        # print("post_quant_conv_mapping_shape", post_quant_conv_mapping.shape)
        
        decoded_images = self.decoder(post_quant_conv_mapping)
        # print("decoded_images_shape", decoded_images.shape)
        return decoded_images, codebook_indices, q_loss
    

    # these functions will be used by the transformers separately.. 
    def encode(self, imgs):
        encoded_images = self.encoder(imgs)
        quant_conv_encoded_images = self.quant_conv(encoded_images)
        codebook_mapping, codebook_indices, q_loss = self.codebook(quant_conv_encoded_images)
        return codebook_mapping, codebook_indices, q_loss

    def decode(self, z):
        post_quant_conv_mapping = self.post_quant_conv(z)
        decoded_images = self.decoder(post_quant_conv_mapping)
        return decoded_images

    # variable lambda is the weighting factor, between vq-vae loss (or perceptual loss)
    # and the GAN loss..  equation 7 in the vq-gan paper
    # G stands for decoder and L stands for the last layer of decoder. 
    def calculate_lambda(self, perceptual_loss, gan_loss):
        
        # last layer and its weight
        last_layer = self.decoder.model[-1]
        last_layer_weight = last_layer.weight
        
        # gradients for both percetpaul and GAN loss. 
        perceptual_loss_grads = torch.autograd.grad(perceptual_loss, last_layer_weight, retain_graph=True)[0]
        gan_loss_grads = torch.autograd.grad(gan_loss, last_layer_weight, retain_graph=True)[0]

        # clipping lambda value between 0 and 10K and then returning lambda. 
        λ = torch.norm(perceptual_loss_grads) / (torch.norm(gan_loss_grads) + 1e-4)
        λ = torch.clamp(λ, 0, 1e4).detach()
        return 0.8 * λ
    
    """
    
    The idea behind starting the discriminator later in the training process is to 
    give the generator (in this case, the VQGAN's encoder-decoder architecture) some 
    time to learn to reconstruct images before introducing the adversarial training 
    aspect. This strategy can help stabilize the training process and improve convergence.

    In the initial phase of training, the generator learns to reconstruct images 
    without the pressure of fooling the discriminator. When the discriminator is 
    introduced later, the generator has already learned a decent representation of
    the data, which helps it better cope with the adversarial training aspect. This can
    lead to better performance and stability during training.
    
    """
    @staticmethod
    def adopt_weight(disc_factor, i, threshold, value=0.):
        if i < threshold:
            disc_factor = value  # disc_factor is the discriminator weight.. 
        return disc_factor

    def load_checkpoint(self, path):
        self.load_state_dict(torch.load(path))










In [ ]:
import os
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import argparse
from tqdm import tqdm
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import utils as vutils
from discriminator import Discriminator
from lpips import LPIPS
# from vqgan_with_pos_trial_v2 import VQGAN
from utils import load_data, weights_init
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Resize, ToTensor
from PIL import Image
import os
import time



class CustomImageDataset(Dataset):
    def __init__(self, main_directory, transform=None):
        self.main_directory = main_directory
        self.transform = transform
        self.image_files = [f for f in os.listdir(main_directory) if os.path.isfile(os.path.join(main_directory, f))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.main_directory, self.image_files[idx])
        
        # # Skip directories
        if os.path.isdir(img_path):
            return None

    
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image
    
class TrainVQGAN:
    def __init__(self, args):
        self.vqgan = VQGAN(args).to(device=args.device)
        self.train_loader = self.load_data_2(args.train_folder, args.batch_size)
        self.valid_loader = self.load_data_2(args.valid_folder, args.batch_size)

        self.discriminator = Discriminator(args).to(device=args.device)
        self.discriminator.apply(weights_init)
        self.perceptual_loss = LPIPS().eval().to(device=args.device)
        self.opt_vq, self.opt_disc = self.configure_optimizers(args)
        self.resumed_epoch = 0
        if args.checkpoint_path is not None:
            self.vqgan.load_state_dict(torch.load(args.checkpoint_path))
            resumed_epoch = int(args.checkpoint_path.split('_')[-1].split('.')[0])
            self.resumed_epoch = resumed_epoch
            print(f"Resuming training from checkpoint: {args.checkpoint_path}")
            print(f"Resuming from epoch {resumed_epoch}")

        if args.checkpoint_vq_opt_path is not None and args.checkpoint_disc_opt_path is not None:
            self.opt_vq.load_state_dict(torch.load(args.checkpoint_vq_opt_path))
            self.opt_disc.load_state_dict(torch.load(args.checkpoint_disc_opt_path))
            
        self.prepare_training()

        self.train(args)

    def load_data_2(self, folder, batch_size):
        transform = Compose([Resize((args.image_size, args.image_size)), ToTensor()])
        dataset = CustomImageDataset(folder, transform=transform)
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)
        

    def configure_optimizers(self, args):
        lr = args.learning_rate
        # vq_vae part optimizer
        opt_vq = torch.optim.Adam(
            list(self.vqgan.encoder.parameters()) +
            list(self.vqgan.decoder.parameters()) +
            list(self.vqgan.codebook.parameters()) +
            list(self.vqgan.quant_conv.parameters()) +
            list(self.vqgan.post_quant_conv.parameters()),
            lr=lr, eps=1e-08, betas=(args.beta1, args.beta2)
        )
        # discriminator part. 
        opt_disc = torch.optim.Adam(self.discriminator.parameters(),
                                    lr=lr, eps=1e-08, betas=(args.beta1, args.beta2))

        return opt_vq, opt_disc

    @staticmethod
    def prepare_training():
        os.makedirs("results_2700Images_with_pos_8192_256", exist_ok=True)
        os.makedirs("checkpoints", exist_ok=True)
        folder_name = args.train_folder
        # os.makedirs(os.path.join("results", folder_name), exist_ok=True)

    def train(self, args):
        # train_dataset = load_data(args)
        # steps_per_epoch = len(train_dataset)
        tensorboard_log_dir = f'runs/lr={args.learning_rate}_with_2700pos_totalepochs={args.epochs}_perceptual_loss_factor={args.perceptual_loss_factor}_discfactor={args.disc_start}_codebook={args.num_codebook_vectors}_latentdim={args.latent_dim}_{time.strftime("%Y%m%d-%H%M%S")}'
        writer = SummaryWriter(log_dir=tensorboard_log_dir)
        steps_per_epoch = len(self.train_loader)
        # output_path = os.path.join(args.train_folder, "results")
        
        
        for epoch in range(self.resumed_epoch+1,args.epochs):
            total_vq_loss = 0  # initialize total_vq_loss
            total_gan_loss = 0  # initialize total_gan_loss
            # The tqdm function is being used here to provide a progress bar for each epoch of training. 
            with tqdm(range(len(self.train_loader))) as pbar:
                
                # In this particular case, the zip function is used to iterate 
                # over both the progress bar and the training dataset simultaneously.
                for i, imgs in enumerate(self.train_loader):
                    # imgs.shape torch.Size([5, 3, 256, 256]) ==> first dimension is batch size(5)
                    # remaining dimension is 3,256,256
                    imgs = imgs.to(device=args.device)
                    decoded_images, _, q_loss = self.vqgan(imgs)

                    disc_real = self.discriminator(imgs)
                    disc_fake = self.discriminator(decoded_images)

                    disc_factor = self.vqgan.adopt_weight(args.disc_factor, epoch*steps_per_epoch+i, threshold=args.disc_start)

                    perceptual_loss = self.perceptual_loss(imgs, decoded_images)
                    rec_loss = torch.abs(imgs - decoded_images)
                    perceptual_rec_loss = args.perceptual_loss_factor * perceptual_loss + args.rec_loss_factor * rec_loss
                    perceptual_rec_loss = perceptual_rec_loss.mean()
                    g_loss = -torch.mean(disc_fake)

                    λ = self.vqgan.calculate_lambda(perceptual_rec_loss, g_loss)
                    vq_loss = perceptual_rec_loss + q_loss + disc_factor * λ * g_loss

                    d_loss_real = torch.mean(F.relu(1. - disc_real))
                    d_loss_fake = torch.mean(F.relu(1. + disc_fake))
                    gan_loss = disc_factor * 0.5*(d_loss_real + d_loss_fake)

                    # Backward loss.. 
                    
                    self.opt_vq.zero_grad()
                    vq_loss.backward(retain_graph=True)

                    self.opt_disc.zero_grad()
                    gan_loss.backward()

                    self.opt_vq.step()
                    self.opt_disc.step()

                    # if i % 10 == 0:
                    #     with torch.no_grad():
                    #         real_fake_images = torch.cat((imgs[:4], decoded_images.add(1).mul(0.5)[:4]))
                    #         vutils.save_image(real_fake_images, os.path.join("results", f"{epoch}_{i}.jpg"), nrow=4)
                    total_vq_loss += vq_loss.item()
                    total_gan_loss += gan_loss.item()
                    
                    pbar.set_postfix(
                        VQ_Loss=np.round(vq_loss.cpu().detach().numpy().item(), 5),
                        GAN_Loss=np.round(gan_loss.cpu().detach().numpy().item(), 3)
                    )
                    pbar.update(0)
                
                # assuming real_fake_images, epoch, i are defined
                # and args is an instance of Args class with train_folder attribute defined
                
                
                with torch.no_grad():
                    real_fake_images = torch.cat((imgs[:4], decoded_images.add(1).mul(0.5)[:4]))
                    vutils.save_image(real_fake_images, os.path.join("results_2700Images_with_pos_8192_256", f"{epoch}_{i}_{args.latent_dim}_{args.num_codebook_vectors}_{args.disc_start}_{args.epochs}.jpg"), nrow=4)
                    
                avg_vq_loss_validation, avg_gan_loss_validation = self.validate(epoch)
                # torch.save(self.vqgan.state_dict(), os.path.join("checkpoints", f"vqgan_epoch_{epoch}.pt"))
                
                avg_vq_loss = total_vq_loss / len(self.train_loader)
                avg_gan_loss = total_gan_loss / len(self.train_loader)
                print(f'Training losses after epoch {epoch}: VQ Loss={avg_vq_loss}, GAN Loss={avg_gan_loss}')

                
                # print(f'Training losses after epoch {epoch}: VQ Loss={avg_vq_loss}, GAN Loss={avg_gan_loss}')
                # # Saving the optimizer state
                # torch.save(self.opt_vq.state_dict(), os.path.join("checkpoints", f"opt_vq_epoch_{epoch}.pt"))
                # torch.save(self.opt_disc.state_dict(), os.path.join("checkpoints", f"opt_disc_epoch_{epoch}.pt"))
                # Log losses and metrics for each training iteration:
                
                if epoch % 100 == 0 and epoch > 0:  # Add condition to save the model and optimizer every 5000 epochs
                    torch.save(self.vqgan.state_dict(), os.path.join("checkpoints", f"vqgan_epoch_{epoch}.pt"))
                    torch.save(self.opt_vq.state_dict(), os.path.join("checkpoints", f"opt_vq_epoch_{epoch}.pt"))
                    torch.save(self.opt_disc.state_dict(), os.path.join("checkpoints", f"opt_disc_epoch_{epoch}.pt"))
                    print(f"Checkpoint saved at epoch {epoch}")
            
            
                writer.add_scalar('Training VQ Loss', vq_loss.item(), epoch*steps_per_epoch + i)
                writer.add_scalar('Training GAN Loss', gan_loss.item(), epoch*steps_per_epoch + i)

                #  Log average losses and metrics for each epoch:
                writer.add_scalar('Validation VQ Loss', avg_vq_loss_validation, epoch*steps_per_epoch + i)
                writer.add_scalar('Validation GAN Loss', avg_gan_loss_validation, epoch*steps_per_epoch + i)

        writer.close()

    def validate(self, epoch):
        with torch.enable_grad():
            total_vq_loss = 0
            total_gan_loss = 0
            # for i, (imgs, _) in enumerate(self.valid_loader):
            for i, imgs in enumerate(self.valid_loader):
                imgs = imgs.to(device=args.device)
                decoded_images, _, q_loss = self.vqgan(imgs)

                disc_real = self.discriminator(imgs)
                disc_fake = self.discriminator(decoded_images)
                disc_factor = self.vqgan.adopt_weight(args.disc_factor, epoch*len(self.valid_loader)+i, threshold=args.disc_start)

                perceptual_loss = self.perceptual_loss(imgs, decoded_images)
                rec_loss = torch.abs(imgs - decoded_images)
                perceptual_rec_loss = args.perceptual_loss_factor * perceptual_loss + args.rec_loss_factor * rec_loss
                perceptual_rec_loss = perceptual_rec_loss.mean()
                g_loss = -torch.mean(disc_fake)

                # with torch.enable_grad():  # enabling gradient computation temporarily
                λ = self.vqgan.calculate_lambda(perceptual_rec_loss, g_loss)
                vq_loss = perceptual_rec_loss + q_loss + disc_factor * λ * g_loss
                # total_vq_loss += vq_loss.item()

                d_loss_real = torch.mean(F.relu(1. - disc_real))
                d_loss_fake = torch.mean(F.relu(1. + disc_fake))
                gan_loss = disc_factor * 0.5*(d_loss_real + d_loss_fake)
                total_gan_loss += gan_loss.item()
                total_vq_loss += vq_loss.item()
                # total_gan_loss += gan_loss.item()
                

            avg_vq_loss_validation = total_vq_loss / len(self.valid_loader)
            avg_gan_loss_validation = total_gan_loss / len(self.valid_loader)
            print(f'Validation losses after epoch {epoch}: VQ Loss={avg_vq_loss_validation}, GAN Loss={avg_gan_loss_validation}')

        return avg_vq_loss_validation, avg_gan_loss_validation    # # Log average losses and metrics for each epoch:
    #     writer.add_scalar('Validation VQ Loss', avg_vq_loss, epoch)
    #     writer.add_scalar('Validation GAN Loss', avg_gan_loss, epoch)    

  





# Just to show how you can use the arguments:
# train_vqgan = TrainVQGAN(args)

class Args:
    def __init__(self,train_folder_name,valid_folder_name, latent_dim,codebook,disc_value):
        
        
        self.latent_dim = latent_dim
        self.image_size = 256
        self.num_codebook_vectors = codebook
        self.beta = 0.25
        self.image_channels = 3
        self.dataset_path = '/data'
        self.device = 'cuda'
        self.batch_size = 5
        self.epochs = 1000
        self.learning_rate = 2.25e-05
        self.beta1 = 0.5
        self.beta2 = 0.9
        self.disc_start = disc_value
        self.disc_factor = 1.
        self.rec_loss_factor = 1.
        self.perceptual_loss_factor = 1
        self.checkpoint_path = None
        self.checkpoint_vq_opt_path = None
        self.checkpoint_disc_opt_path = None
        self.train_folder = os.path.join(os.getcwd(), train_folder_name)
        self.valid_folder = os.path.join(os.getcwd(), valid_folder_name)
        # Create a unique name for the tensorboard log directory
        

        # Define the folder names
train_folder_name = ['train_all_only_few','train_all', 'train_all_8K']
valid_folder_name = ['valid_all_only_few','valid_all', 'valid_all_8K']

train_folder_name = ['train_all']
valid_folder_name = ['valid_all']

codebook_vector = [8192]
latent_dims = [256]
disc_starts = [100]
# perceptual_loss_factors = [0.5]
# rec_loss_factors = [0.5]
# for i in range(1,len(train_folder_name)):
for i in range(1):
    
    for latent_dim in latent_dims:
        for codebook in codebook_vector:
            for disc_value in disc_starts:
                args = Args(train_folder_name[i], valid_folder_name[i], latent_dim,codebook, disc_value)



                # Just to show how you can use the arguments:
                train_vqgan = TrainVQGAN(args)

  0%|          | 0/297 [01:18<?, ?it/s, GAN_Loss=0, VQ_Loss=0.316]    


Validation losses after epoch 1: VQ Loss=0.21269162041800363, GAN Loss=0.006155002183680023
Training losses after epoch 1: VQ Loss=0.2888010829586774, GAN Loss=0.10053967336652901


  0%|          | 0/297 [01:18<?, ?it/s, GAN_Loss=0.117, VQ_Loss=0.455]


Validation losses after epoch 2: VQ Loss=0.3961883762053081, GAN Loss=0.031239473802686137
Training losses after epoch 2: VQ Loss=0.47368105533548477, GAN Loss=0.013141139748579852


  0%|          | 0/297 [01:19<?, ?it/s, GAN_Loss=0, VQ_Loss=1.22]     


...

  0%|          | 0/297 [01:17<?, ?it/s, GAN_Loss=0, VQ_Loss=0.025]      


Validation losses after epoch 328: VQ Loss=0.04854484977466719, GAN Loss=0.0019321102524112836
Training losses after epoch 328: VQ Loss=0.038378467002526076, GAN Loss=0.006748675527762656


  0%|          | 0/297 [01:17<?, ?it/s, GAN_Loss=0, VQ_Loss=0.0584]     


Validation losses after epoch 329: VQ Loss=0.04470530500901597, GAN Loss=0.0074074620372552346
Training losses after epoch 329: VQ Loss=0.04247715845484575, GAN Loss=0.006451498886599285


  0%|          | 0/297 [00:21<?, ?it/s, GAN_Loss=0, VQ_Loss=0.0556]    

In [5]:
import shutil
shutil.make_archive('checkpoints', 'zip', 'checkpoints')

'/home/verma.lu/VQGANs/checkpoints.zip'

In [1]:
!zip -r checkpoints.zip checkpoints

updating: checkpoints/ (stored 0%)
  adding: checkpoints/opt_disc_epoch_100.pt (deflated 8%)
  adding: checkpoints/opt_disc_epoch_200.pt (deflated 7%)
  adding: checkpoints/opt_disc_epoch_300.pt (deflated 7%)
  adding: checkpoints/opt_disc_epoch_without_pos_300.pt (deflated 7%)
  adding: checkpoints/opt_disc_epoch_without_pos_5000.pt (deflated 8%)
  adding: checkpoints/opt_vq_epoch_100.pt (deflated 8%)
  adding: checkpoints/opt_vq_epoch_200.pt (deflated 11%)
  adding: checkpoints/opt_vq_epoch_300.pt (deflated 11%)
  adding: checkpoints/opt_vq_epoch_without_pos_300.pt (deflated 10%)
  adding: checkpoints/opt_vq_epoch_without_pos_5000.pt (deflated 11%)
  adding: checkpoints/vqgan_epoch_100.pt (deflated 10%)
  adding: checkpoints/vqgan_epoch_200.pt (deflated 10%)
  adding: checkpoints/vqgan_epoch_300.pt (deflated 10%)
  adding: checkpoints/vqgan_epoch_without_pos_300.pt (deflated 8%)
  adding: checkpoints/vqgan_epoch_without_pos_5000.pt (deflated 8%)


In [2]:
!zip -r results_65Images_without_pos_pruned_v1.zip results_65Images_without_pos_pruned_v1

  adding: results_65Images_without_pos_pruned_v1/ (stored 0%)
  adding: results_65Images_without_pos_pruned_v1/.ipynb_checkpoints/ (stored 0%)
  adding: results_65Images_without_pos_pruned_v1/.ipynb_checkpoints/1008_12_256_8192_100_5001-checkpoint.jpg (deflated 1%)
  adding: results_65Images_without_pos_pruned_v1/.ipynb_checkpoints/101_12_256_8192_100_5001-checkpoint.jpg (deflated 1%)
  adding: results_65Images_without_pos_pruned_v1/.ipynb_checkpoints/1_12_256_8192_100_5001-checkpoint.jpg (deflated 10%)
  adding: results_65Images_without_pos_pruned_v1/.ipynb_checkpoints/223_12_256_8192_100_5001-checkpoint.jpg (deflated 1%)
  adding: results_65Images_without_pos_pruned_v1/.ipynb_checkpoints/226_12_256_8192_100_5001-checkpoint.jpg (deflated 1%)
  adding: results_65Images_without_pos_pruned_v1/.ipynb_checkpoints/235_12_256_8192_100_5001-checkpoint.jpg (deflated 1%)
  adding: results_65Images_without_pos_pruned_v1/.ipynb_checkpoints/2593_12_256_8192_100_5001-checkpoint.jpg (deflated 1%)
  

In [3]:
!zip -r results_2700Images_with_pos_1024_256.zip results_2700Images_with_pos_1024_256

  adding: results_2700Images_with_pos_1024_256/ (stored 0%)
  adding: results_2700Images_with_pos_1024_256/.ipynb_checkpoints/ (stored 0%)
  adding: results_2700Images_with_pos_1024_256/.ipynb_checkpoints/16_296_256_1024_100_1000-checkpoint.jpg (deflated 2%)
  adding: results_2700Images_with_pos_1024_256/.ipynb_checkpoints/17_296_256_1024_100_1000-checkpoint.jpg (deflated 1%)
  adding: results_2700Images_with_pos_1024_256/.ipynb_checkpoints/229_296_256_1024_100_1000-checkpoint.jpg (deflated 1%)
  adding: results_2700Images_with_pos_1024_256/.ipynb_checkpoints/239_296_256_1024_100_1000-checkpoint.jpg (deflated 4%)
  adding: results_2700Images_with_pos_1024_256/.ipynb_checkpoints/266_296_256_1024_100_1000-checkpoint.jpg (deflated 1%)
  adding: results_2700Images_with_pos_1024_256/.ipynb_checkpoints/2_296_256_1024_100_1000-checkpoint.jpg (deflated 3%)
  adding: results_2700Images_with_pos_1024_256/.ipynb_checkpoints/333_296_256_1024_100_1000-checkpoint.jpg (deflated 1%)
  adding: results_

In [4]:
!zip -r results_2700Images_with_pos_8192_256.zip results_2700Images_with_pos_8192_256

  adding: results_2700Images_with_pos_8192_256/ (stored 0%)
  adding: results_2700Images_with_pos_8192_256/.ipynb_checkpoints/ (stored 0%)
  adding: results_2700Images_with_pos_8192_256/.ipynb_checkpoints/129_296_256_8192_100_1000-checkpoint.jpg (deflated 1%)
  adding: results_2700Images_with_pos_8192_256/.ipynb_checkpoints/137_296_256_8192_100_1000-checkpoint.jpg (deflated 1%)
  adding: results_2700Images_with_pos_8192_256/.ipynb_checkpoints/184_296_256_8192_100_1000-checkpoint.jpg (deflated 1%)
  adding: results_2700Images_with_pos_8192_256/.ipynb_checkpoints/19_296_256_8192_100_1000-checkpoint.jpg (deflated 1%)
  adding: results_2700Images_with_pos_8192_256/.ipynb_checkpoints/227_296_256_8192_100_1000-checkpoint.jpg (deflated 1%)
  adding: results_2700Images_with_pos_8192_256/.ipynb_checkpoints/228_296_256_8192_100_1000-checkpoint.jpg (deflated 2%)
  adding: results_2700Images_with_pos_8192_256/.ipynb_checkpoints/249_296_256_8192_100_1000-checkpoint.jpg (deflated 1%)
  adding: resul

In [5]:
!zip -r results_65Images_without_pos.zip results_65Images_without_pos

  adding: results_65Images_without_pos/ (stored 0%)
  adding: results_65Images_without_pos/.ipynb_checkpoints/ (stored 0%)
  adding: results_65Images_without_pos/.ipynb_checkpoints/1829_12_256_8192_100_10001-checkpoint.jpg (deflated 1%)
  adding: results_65Images_without_pos/.ipynb_checkpoints/2099_12_256_8192_100_10001-checkpoint.jpg (deflated 1%)
  adding: results_65Images_without_pos/.ipynb_checkpoints/3_12_256_8192_100_10001-checkpoint.jpg (deflated 7%)
  adding: results_65Images_without_pos/.ipynb_checkpoints/4309_12_256_8192_100_10001-checkpoint.jpg (deflated 1%)
  adding: results_65Images_without_pos/.ipynb_checkpoints/4310_12_256_8192_100_10001-checkpoint.jpg (deflated 1%)
  adding: results_65Images_without_pos/.ipynb_checkpoints/4311_12_256_8192_100_10001-checkpoint.jpg (deflated 0%)
  adding: results_65Images_without_pos/.ipynb_checkpoints/449_12_256_8192_100_10001-checkpoint.jpg (deflated 1%)
  adding: results_65Images_without_pos/.ipynb_checkpoints/451_12_256_8192_100_10001

In [6]:
!zip -r runs.zip runs

updating: runs/ (stored 0%)
updating: runs/.ipynb_checkpoints/ (stored 0%)
updating: runs/lr=2.25e-05_codebook=1024_latentdim=256_20230720-022507/ (stored 0%)
updating: runs/lr=2.25e-05_codebook=1024_latentdim=256_20230720-022507/events.out.tfevents.1689834307.d1028.31210.7 (deflated 71%)
updating: runs/lr=2.25e-05_codebook=1024_latentdim=512_20230720-040938/ (stored 0%)
updating: runs/lr=2.25e-05_codebook=1024_latentdim=512_20230720-040938/events.out.tfevents.1689840578.d1028.31210.8 (deflated 73%)
updating: runs/lr=2.25e-05_perceptual_loss_factor=1_codebook=1024_latentdim=128_20230720-185322/ (stored 0%)
updating: runs/lr=2.25e-05_perceptual_loss_factor=1_codebook=1024_latentdim=128_20230720-185322/events.out.tfevents.1689893602.d1028.64782.1 (deflated 71%)
updating: runs/lr=2.25e-05_perceptual_loss_factor=1_codebook=1024_latentdim=256_20230720-223543/ (stored 0%)
updating: runs/lr=2.25e-05_perceptual_loss_factor=1_codebook=1024_latentdim=256_20230720-223543/events.out.tfevents.16899

In [7]:
import shutil
import os

# Define the directory name
dir_name = "checkpoints"

# Check if the directory exists
if os.path.exists(dir_name):
    # Remove the directory and all its contents
    shutil.rmtree(dir_name)
else:
    print(f"The directory {dir_name} does not exist")

In [17]:
# !pip install albumentations

# !pip install matplotlib

In [23]:
import os

current_path = os.getcwd()
print(current_path)

/home/verma.lu/VQGANs


In [19]:
%tb

SystemExit: 2